In [ ]:
!wget https://os.unil.cloud.switch.ch/fma/fma_small.zip
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip

!unzip fma_small.zip
!unzip fma_metadata.zip

In [ ]:
import librosa
import pandas as pd
from pathlib import Path

# 1. Load Metadata
tracks = pd.read_csv('fma_metadata/tracks.csv', index_col=0, header=[0, 1])

# 2. Define the Production Loader
class FMALoader:
    def __init__(self, audio_dir):
        self.audio_dir = Path(audio_dir)

    def get_path(self, track_id):
        # FMA pads IDs with zeros: 2 -> 000002.mp3
        # Folder structure is first 3 digits: 000/000002.mp3
        tid_str = '{:06d}'.format(track_id)
        return self.audio_dir / tid_str[:3] / f'{tid_str}.mp3'

    def load_track(self, track_id):
        path = self.get_path(track_id)
        if not path.exists():
            return None, None

        try:
            # Load 10s only for quick verify
            y, sr = librosa.load(path, sr=22050, duration=10)
            return y, sr
        except Exception as e:
            print(f"Error: {e}")
            return None, None

# 3. Run the Check
loader = FMALoader('fma_small')
track_id = 2  # The famous 'Food' track by AWOL
y, sr = loader.load_track(track_id)

if y is not None:
    print(f"Track: {tracks.loc[track_id, ('track', 'title')]}")
    print(f"Genre: {tracks.loc[track_id, ('track', 'genre_top')]}")
    print(f"Audio Shape: {y.shape} (Sample Rate: {sr})")
else:
    print("\n PIPELINE BROKEN - File still not found.")